In [ ]:
import torch, warnings, sys
import numpy as np, pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneGroupOut
from config import cols, labels
from train_functions.MLPLogistic import MLPLogistic
import train_functions.EMONET as em
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

type = 'Wesad'
df = pd.read_csv(f'./datas/{type}.csv',index_col=0)
logo = LeaveOneGroupOut()
groups = df['pnum']
unique_groups = np.unique(groups)
group_count = len(unique_groups)

def apply_voting(arr, window=10):
    usable_len = (len(arr) // window) * window  # 10의 배수로 자르기
    arr = arr[:usable_len].reshape(-1, window)
    return np.array([np.bincount(group).argmax() for group in arr])

def test_run() :
    accs2 = []
    accs = []
    for i,(train_idx, test_idx) in enumerate(logo.split(df, df[labels[type]], groups)):
        x_data = df.drop(columns=cols[type])
        y_data = df[labels[type]]
        x_train, x_test = np.array(x_data.iloc[train_idx]), np.array(x_data.iloc[test_idx])
        y_train, y_test = np.array(y_data.iloc[train_idx]).reshape(-1,1), np.array(y_data.iloc[test_idx]).reshape(-1,1)
        print(x_train.shape,y_train.shape)
        indices = np.arange(x_train.shape[0])

        indices = np.arange(x_test.shape[0])
        accs2.append(em.train_cnn_svm_pipeline(x_train, y_train, x_test, y_test))
        model = MLPLogistic(x_train.shape[1], 32, 1400, 5e-2, np.random.randint(10000))
        model.fit(x_train, y_train)

        proba = model.forward(torch.Tensor(x_test))
        pred = np.where(proba>=0.5,1,0)

        voted_pred = apply_voting(pred)
        voted_y = apply_voting(y_test.reshape(-1))
        
        accs.append(accuracy_score(voted_y, voted_pred))

        bar = '[' + '=' * int((i+1) * 31 / group_count) + ' ' * (31 - int((i+1) * 31 / group_count)) + ']'
        percent = ((i+1) / group_count) * 100
        sys.stdout.write(f'\rProgress: {bar} {percent:.2f}% ({(i+1)}/{group_count})')
        sys.stdout.flush()
    return accs,accs2

In [ ]:
ret1,ret2 = [],[]
for i in range(5) :
    a1,a2 = test_run()
    ret1.append(a1)
    ret2.append(a2)